# RAG - PDF Q&A Using GPT

## 1) Importing Required Modules

In [1]:
!pip install langchain openai chromadb tiktoken pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 118.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.4 MB/s eta 0

In [1]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17


In [2]:
# Import required modules
from langchain import hub
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "API_KEY"

In [ ]:
llm = OpenAI()
print(llm("Qual é a capital do Brasil?"))

<ipython-input-5-299eba925df6>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI()
<ipython-input-5-299eba925df6>:2: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm("Qual é a cpaital do Brasil?"))




A capital do Brasil é Brasília.


## 2) Document Loading

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
loader = PyPDFLoader("/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf")
data = loader.load()

In [6]:
len(data)

73

In [ ]:
page = data[3]

In [ ]:
print(page.page_content[:500])

 
III – de Conselheiro, Auditor ou Procurador do Ministério Públic o, no Tribunal 
de Contas;  
IV – (V E T A D O). 
§ 1º As vedações deste artigo aplicam-se: 
I – aos casos de reciprocidade de nomeação ou designação; 
II – às relações homoafetivas. 
§ 2º Não se inclui nas vedações deste artigo a nomeação ou a designação: 
I – de servidor ocupante de cargo de provimento efetivo, incluídos os 
aposentados, desde que seja observada: 
a) a compatibilidade do grau de escolaridade do cargo efetivo co


In [ ]:
page.metadata

{'source': '/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf',
 'page': 3,
 'page_label': '4'}

## 4) Splitting Document Text

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3500, chunk_overlap=0) #default separator = ["\n\n", "\n", " ", ""]

all_splits = text_splitter.split_documents(data)

In [8]:
len(all_splits)

73

## 5) Vectorstores and Embeddings

In [17]:
persist_directory = 'data'

vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=OpenAIEmbeddings(model='text-embedding-3-small'),
    persist_directory=persist_directory
)

In [10]:
print(vectorstore._collection.count())

73


## 6) Retriever

Esse comando é baseado na seguinte documentação: https://python.langchain.com/docs/how_to/vectorstore_retriever/

In [11]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":5}) # transformar um repositório de vetores em um retriever

Existem diversos outros recuperadores de texto disponíveis no LangChain: https://python.langchain.com/docs/integrations/retrievers/

### 6.1 Teste: Pesquisa de Similaridade

In [ ]:
question = "Sou servidora pública. O que devo receber antes de tirar as minhas férias?"

vectorstore.similarity_search(question, k=2)

[Document(metadata={'page': 27, 'page_label': '28', 'source': '/content/drive/MyDrive/RAG/Lei-Complementar-8402011.pdf'}, page_content='pública, as férias podem ser parceladas em até três períodos, nenhum deles inferior a \ndez dias.  \nArt. 126. Até dois dias antes de as férias serem iniciadas, devem ser pagos a o \nservidor:  \nI – o adicional de férias;  \nII – o abono pecuniário, se deferido;  \nIII – o adiantamento de parcela correspondente a quarenta por cento do valor \nlíquido do subsídio ou remuneração, desde que requerido.  \nParágrafo único. O adiantamento de que tr ata o inciso III é descontado do \nsubsídio ou remuneração do servidor em quatro parcelas mensais e sucessivas de \nidêntico valor.  \nArt. 127.  O servidor que opera direta e permanentemente com raios X ou \nsubstâncias radioativas tem de gozar vinte dias consecut ivos de férias, por semestre de \natividade profissional, proibida em qualquer hipótese a acumulação.  \nParágrafo único. O servidor referido neste ar

### 6.2 Teste: Maximum Marginal Relevance

In [ ]:
vectorstore.max_marginal_relevance_search(question, k=3)

### 6.5 Teste: TFIDF

In [ ]:
# TESTE E TFIDF RETRIEVER
from langchain.retrievers import TFIDFRetriever

tfidf_retriever = TFIDFRetriever.from_documents(all_splits)

In [ ]:
docs_tfidf = tfidf_retriever.get_relevant_documents(question)
pretty_print_docs(docs_tfidf)

Document 1:

67 
 
Indenização de transporte 
O que é: É a indenização devida ao servidor que realiza despesas com a utilização de 
meio próprio de locomoção para a execução de serviços externos, por força das 
atribuições próprias do cargo. 
Quem pode utilizar este serviço: Servidor efetivo.  
 Meio de solicitação: Processo SEI.  
Tipo de processo – Pessoal: indenização de transporte 
Tipo de documento: Requerimento 
Para a Subcontroladoria de Controle Interno: POP – indenização de transporte.  
Unidade responsável: Gerência de Registros Financeiros – GERFI/ DITEC/COGEP 
(2108-3249) Diretoria Técnica de Gestão de Pessoas – DITEC/COGEP (2108-3381) 
Observações:  
- Deverá haver lei específica autorizando o pagamento; 
- Deverá haver publicação de Ordem de Serviço especificando o local e serviço; 
- Não há pagamento de indenização de transporte para os dias de afastamentos, 
licenças, férias, feriados, ponto facultativo etc. 
- Deverá estar atestado pelo chefe imediato. 
- A carreira Au

## 7) Defining Prompt Template and Memory Handling

In [13]:
template = """
Você é um assistente especializado em recuperação de informações. Sua resposta deve ser **clara, objetiva e baseada exclusivamente nos documentos fornecidos**. **Não invente informações.** Responda **apenas em português**.

Se o contexto fornecido **não contiver informações suficientes para responder com precisão**, diga que não possui dados suficientes em vez de tentar adivinhar.

Cite os artigos em que as respostas foram baseadas.

### **Contexto:**
{context}

### **Histórico da Conversa:**
{history}

### **Pergunta do Usuário:**
{question}

### **Resposta do Assistente (em português):**
"""


In [14]:
prompt = PromptTemplate(
        input_variables=["history", "context", "question"],
        template=template
    )

In [15]:
memory = ConversationBufferMemory(
        memory_key="history",
        return_messages=True,
        input_key="question"
    )

<ipython-input-15-bc0b13bb1ee8>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


## 8) Configuring RetrievalQA Chain

In [20]:
!pip install langchain-openai==0.2.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 4.8 MB/s eta 0:00:00


In [21]:
from langchain_openai import ChatOpenAI

qa_chain = RetrievalQA.from_chain_type(
            llm= ChatOpenAI(model_name="gpt-4o-mini", temperature=0),
            chain_type='stuff',
            retriever=retriever,
            verbose=False,
            chain_type_kwargs={
                "verbose": True,
                "prompt": prompt,
                "memory": memory,
            }
        )

## 9) Setting Up Query and Invoking Q&A Chain

In [26]:
query = "Como servidora pública do GDF, faço jus a quais tipos de licença?"
query += ".Somente deste PDF. Seja breve."

In [27]:
qa_chain.invoke({"query": query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Você é um assistente especializado em recuperação de informações. Sua resposta deve ser **clara, objetiva e baseada exclusivamente nos documentos fornecidos**. **Não invente informações.** Responda **apenas em português**.

Se o contexto fornecido **não contiver informações suficientes para responder com precisão**, diga que não possui dados suficientes em vez de tentar adivinhar.

Cite os artigos em que as respostas foram baseadas.

### **Contexto:**
I – as férias;  
II – as ausências previstas no art. 62; 
III – a licença:  
a) maternidade ou paternidade;  
b) médica ou odontológica;  
c) prêmio por assiduidade;  
d) para o serviço militar obrigatório; 
IV – o abono de ponto;  
V – o afastamento para:  
a) exercício em outro órgão ou entidade, inclusive em cargo em comissão ou 
função de confiança, de qualquer dos Poderes do Distrito Federal, União, Estado ou 
Município;  
b) es

{'query': 'Como servidora pública do GDF, faço jus a quais tipos de licença?.Somente deste PDF. Seja breve.',
 'result': 'Como servidora pública do GDF, você faz jus às seguintes licenças:\n\nI – Licença maternidade ou paternidade;  \nII – Licença médica ou odontológica;  \nIII – Licença prêmio por assiduidade;  \nIV – Licença para serviço militar obrigatório;  \nV – Licença para desempenho de mandato classista;  \nVI – Licença por motivo de doença em pessoa da família;  \nVII – Licença para atividade política;  \nVIII – Licença para o desempenho de mandato eletivo federal, estadual, distrital ou municipal.\n\nEssas informações estão baseadas nos artigos 134 a 150 e 145 a 149 da Lei Complementar nº 840 de 23 de dezembro de 2011.'}

In [28]:
query = "O que é cargo público?"
query += ".Somente deste PDF. Seja breve."

qa_chain.invoke({"query": query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Você é um assistente especializado em recuperação de informações. Sua resposta deve ser **clara, objetiva e baseada exclusivamente nos documentos fornecidos**. **Não invente informações.** Responda **apenas em português**.

Se o contexto fornecido **não contiver informações suficientes para responder com precisão**, diga que não possui dados suficientes em vez de tentar adivinhar.

Cite os artigos em que as respostas foram baseadas.

### **Contexto:**
LEI COMPLEMENTAR Nº 840 DE 23 DE DEZEMBRO DE 2011.  
(Autoria do Projeto: Poder Executivo) 
 
Dispõe sobre o regime jurídico dos 
servidores públicos civis do Distrito Federal, 
das autarquias e das  fundações públicas 
distritais. 
 
O GOVERNADOR DO DISTRITO FEDE RAL, FAÇO SABER QUE A CÂMARA 
LEGISLATIVA DO DISTRITO FEDERAL DECRETA E EU SANCIONO A SEGUINTE LEI: 
TÍTULO I 
CAPÍTULO ÚNICO 
DAS DISPOSIÇÕES PRELIMINARES 
Art. 1º Esta Le

{'query': 'O que é cargo público?.Somente deste PDF. Seja breve.',
 'result': 'Cargo público é o conjunto de atribuições e responsabilidades previstas na estrutura organizacional e cometidas a um servidor público. Os cargos públicos são criados por lei, com denominação própria e subsídio ou vencimentos pagos pelos cofres públicos, para provimento em caráter efetivo ou em comissão. (Art. 3º da Lei Complementar nº 840 de 23 de dezembro de 2011).'}

In [29]:
query = "Um servidor pode se afastar do cargo ocupado para participar de curso de formação?"
query += ".Somente deste PDF. Seja breve."
qa_chain.invoke({"query": query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Você é um assistente especializado em recuperação de informações. Sua resposta deve ser **clara, objetiva e baseada exclusivamente nos documentos fornecidos**. **Não invente informações.** Responda **apenas em português**.

Se o contexto fornecido **não contiver informações suficientes para responder com precisão**, diga que não possui dados suficientes em vez de tentar adivinhar.

Cite os artigos em que as respostas foram baseadas.

### **Contexto:**
ou mediante compensação de horário, afastar -se do exercício do cargo efetivo, com a 
respectiva remuneração ou subsídio, para participar de programa de pós -graduação 
stricto sensu em instituição de ensino superior, no País ou no exterior.  
§ 1º O titular do órgão, autarquia ou fundação deve definir os programas de 
capacitação e os critérios para participação em programas de pós -graduação de que 
trata este artigo, com ou sem af

{'query': 'Um servidor pode se afastar do cargo ocupado para participar de curso de formação?.Somente deste PDF. Seja breve.',
 'result': 'Sim, um servidor pode se afastar do cargo ocupado para participar de curso de formação, desde que haja expressa previsão do curso no edital do concurso e incompatibilidade entre os horários das aulas e os da repartição. (Art. 162).'}

In [ ]:
query = "Havendo incompatibilidade entre os horários, o que ocorre?"
query += ".Somente deste PDF. Seja breve."

In [ ]:
qa_chain.invoke({"query": query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Você é um assistente especializado em recuperação de informações. Sua resposta deve ser **clara, objetiva e baseada exclusivamente nos documentos fornecidos**. **Não invente informações.** Responda **apenas em português**.

Se o contexto fornecido **não contiver informações suficientes para responder com precisão**, diga que não possui dados suficientes em vez de tentar adivinhar.

### **Contexto:**
públicas, o Governador pode autorizar, excepcionalmente, a extrapolação dos limites 
previstos neste artigo, para os servidores que atuem diretamente nas áreas envolvidas.  
Art. 61. Pode ser concedido horário especial:  
I – ao servidor com deficiên cia, quando comprovada a necessidade por junta 
médica oficial;  
II – ao servidor que tenha cônjuge, filho ou dependente com deficiência;  
III – ao servidor matriculado em curso da educação básica e da educação 
superior, quando comprova

{'query': 'Havendo incompatibilidade ntre os horários, o que ocorre?.Somente deste PDF. Seja breve.',
 'result': '\nCaso haja incompatibilidade entre os horários do servidor e o curso de formação previsto como etapa do concurso público, ele pode requerer autorização do chefe imediato para se afastar do cargo ocupado e participar do curso, conforme o artigo 162 da Lei Complementar 840/2011.'}

In [ ]:
query = "Como ficará a sua remuneração durante o curso de formação?"
query += ".Somente deste PDF. Seja breve."

In [ ]:
qa_chain.invoke({"query": query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Você é um assistente especializado em recuperação de informações. Sua resposta deve ser **clara, objetiva e baseada exclusivamente nos documentos fornecidos**. **Não invente informações.** Responda **apenas em português**.

Se o contexto fornecido **não contiver informações suficientes para responder com precisão**, diga que não possui dados suficientes em vez de tentar adivinhar.

### **Contexto:**
II – sem remuneração, nos casos de curso de formação para cargo não 
contemplado no inciso I deste parágrafo. 
§ 2º O servidor pode optar por eventual ajuda financeira paga em razão do 
curso de formação, vedada a percepção da remuneração prevista no § 1º, I.   
CAPÍTULO V 
DO TEMPO DE SERVIÇO E DO TEMPO DE CONTRIBUIÇÃO 
Seção I 
Do Tempo de Serviço 
Art. 163. Salvo disposição legal em contrário, é contado para todos os efeitos 
o tempo de serviço público remun erado, prestado a órgão,

{'query': 'Como ficará a sua remuneração durante o curso de formação?.Somente deste PDF. Seja breve.',
 'result': 'Durante o curso de formação, você não receberá remuneração. No entanto, caso opte por receber ajuda financeira, não poderá receber a remuneração prevista no § 1º, I da Lei Complementar 840/2011.'}

In [ ]:
query = "O servidor afastado não recebe remuneração ou subsídios nem em casos de de curso de formação para cargo efetivo de órgão, autarquia ou fundação do Distrito Federal?"
query += ".Somente deste PDF. Seja breve."

In [ ]:
qa_chain.invoke({"query": query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Você é um assistente especializado em recuperação de informações. Sua resposta deve ser **clara, objetiva e baseada exclusivamente nos documentos fornecidos**. **Não invente informações.** Responda **apenas em português**.

Se o contexto fornecido **não contiver informações suficientes para responder com precisão**, diga que não possui dados suficientes em vez de tentar adivinhar.

### **Contexto:**
Legislativa ou do Presidente do Tribunal de Contas, o servidor estável pode ausentar -se 
do Distrito Federal ou do País para: 
I – estudo ou missão oficial, com a remuneração ou subsídio do cargo efetivo; 
II – serviço sem remuneração em organismo internacional de que o Brasil 
participe ou com o qual coopere. 
§ 1º A ausência não pode exceder a quatro anos, nem pode ser concedida nova 
licença antes de decorrido igual período. 
§ 2º Em caso de exoneração, demissão, aposentadoria volu

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 4192 tokens (3936 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

## Evaluation Metrics


In [ ]:
memory.clear()

In [30]:
!pip install datasets
from datasets import Dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 w

In [32]:
questions = ["Qual é o percentual aplicado pela Lei Complementar 840/2011 para reserva de vagas às pessoas portadoras de deficiência?",
             "A realização de concurso público é obrigatório para que?",
             "Quais fatores são observados durante o estágio probatório de um servidor público?",
             "Tenho direito a folga para consulta ou exames de rotina?",
             "Para fazer mestrado ou doutorado, tenho direito a horário diferenciado ou licença?",
             "Fui aprovado em concurso público federal, tendo direito a licença remunerada?",
             "O que acontece se eu acumular dois cargos públicos?",
             "Estou próximo a me aposentar, tenho alguma restrição?",
             "Posso ter um outro trabalho na esfera privada?",
             "Como é a licença prêmio/servidor?"
            ]

ground_truths = [["O percentual aplicado para reserva de vagas às pessoas portadoras de deficiência é de 20%"],
                ["A realização de concurso público é obrigatório para provimento de cargo efetivo e emprego público"],
                ["São observadas a responsabilidade, a assiduidade, a capacidade de iniciativa e a produditividades."],
                ["Sem prejuízo da remuneração ou subsídio, o servidor pode ausentar-se do serviço, mediante comunicação prévia à chefia imediata: por um dia para realizar, uma vez por ano, exames médicos preventivos ou periódicos voltados ao controle de câncer de próstata, de mama ou do colo de útero"],
                ["Pode ser concedido horário especial ao servidor matriculado em curso da educação básica e da educação superior, quando comprovada a incompatibilidade entre o horário escolar e o da unidade administrativa, sem prejuízo do exercício do cargo. É exigida do servidor a compensação de horário na unidade administrativa, de modo a cumprir integralmente o regime semanal de trabalho. O servidor estudante deve comprovar, mensalmente, a sua frequência escolar. Sobre o afastamento para participar de programa de Pós-Graduação Strictu Sensu, o servidor estável pode, no interesse da administração pública, e desde que a partici­pação não possa ocorrer simultaneamente com o exercício do cargo ou mediante compensação horário, afastar-se do exercício do cargo efetivo, com a respectiva remuneração ou subsídio, para participar de programa de pós-graduação stricto sensu em instituição de ensino superior, no País ou no exterior. O afastamento para realização de programas de mestrado, doutorado ou pós-doutorado somente pode ser concedido ao servidor estável que esteja em efetivo exercício no respectivo órgão, autarquia ou fundação há pelo menos três anos consecutivos para mestrado e quatro anos consecutivos para doutorado ou pós-doutorado."],
                ["Pode gozar de afastamento caso haja expressa previsão do curso de formação no edital do concurso e incompatibilidade entre os horários das aulas e os da repartição. O afastamento só pode ser com remuneração ou subsídio, nos casos de curso de formação para cargo efetivo de órgão, autarquia ou fundação dos Poderes Legislativo ou Executivo do Distrito Federal."],
                ["A Lei Complementar nº 840/2011 permite a acumulação de cargos públicos apenas nas hipóteses previstas na Constituição Federal (art. 37, XVI), como:a) dois cargos de professor; b) um cargo de professor e outro técnico ou científico; c) dois cargos ou empregos privativos de profissionais de saúde. Se um servidor acumular cargos de forma irregular, ele pode ser punido com demissão. O processo ocorre por meio de um PAD, que assegura o direito ao contraditório e à ampla defesa antes da aplicação da penalidade."],
                ["Algumas possíveis restrições: a) é necessário que o servidor tenha completado o tempo de contribuição e a idade exigidos por lei para se aposentar voluntariamente; b) apenas servidores efetivos podem solicitar a aposentadoria voluntária, c) se um servidor estiver respondendo a um Processo Administrativo Disciplinar (PAD) que possa resultar em demissão, seu pedido de aposentadoria poderá ser negado até a conclusão do processo; d) caso o servidor tenha sofrido penalidades graves, como demissão por infração disciplinar, pode perder o direito à aposentadoria estatutária; e) se o servidor tiver se ausentado por afastamento para estudo ou missão no exterior ou para participar de programa de Pós-Graduação Stricto Sensu e aposentar antes de decorrido período igual ao do afastamento, o servidor de ressarcir proporcionalmente a despesa, incluída a remuneração ou o subsídio e encargos sociais."],
                ["A Lei Complementar nº 840/2011 não proíbe o exercício de atividade remunerada no setor privado, desde que não haja conflito de interesses com o cargo público e que a atividade não comprometa a jornada de trabalho do servidor. São consideradas infrações: exercer atividade privada incompatível com o horário do serviço; exercer atividade privada incompatível com o exercício do cargo público ou da função de confiança; participar de gerência ou administração de sociedade ou empresa privada, personificada ou não personificada, salvo: a) nos casos previstos nesta Lei Complementar; b) nos períodos de licença ou afastamento do cargo sem remuneração, desde que não haja proi­bição em sentido contrário, nem incompatibilidade; c) em instituições ou entidades beneficentes, filantrópicas, de caráter social e humanitário e sem fins lucrativos, quando compatíveis com a jornada de trabalho."],
                ["A licença prêmio foi substituída pela licença servidor pela Lei Complementar 952/2019. Após cada quinquênio de efetivo exercício, o servidor ocupante de cargo efetivo faz jus a 3 meses de licença-servidor, sem prejuízo de sua remuneração, inclusive da retribuição do cargo em comissão, função de confiança ou função gratificada escolar - FGE que eventualmente exerça. Os períodos não são acumuláveis e não podem ser convertidos em pecúnia. O número de servidores afastados em virtude de licença-servidor não pode ser superior a 1/3 da lotação da respectiva unidade administrativa do órgão, autarquia ou fundação. A administração tem o prazo de até 120 dias, contado da data de requerimento do pedido pelo servidor, para definir o período de gozo da licença. Fica assegurado às servidoras e aos servidores o direito de iniciar a fruição de licença-servidor logo após o término da licença-maternidade ou da licença-paternidade."]]

answers = []
contexts = []
references = []

# Inference
for query in questions:
  # Extract the generated text from the LLM response
  response = qa_chain.invoke(query)
  answers.append(response['result'])  # Assuming 'result' key holds the text
  contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])
  # **Add the ground truth as the reference here**
  ground_truth_for_query = ground_truths[questions.index(query)]
  references.append(ground_truth_for_query) # Assuming you have a list of references

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    # **Remove 'ground_truths', keep 'reference' as a string**
    "reference": [item[0] for item in ground_truths]  # Extract the string from the list
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Você é um assistente especializado em recuperação de informações. Sua resposta deve ser **clara, objetiva e baseada exclusivamente nos documentos fornecidos**. **Não invente informações.** Responda **apenas em português**.

Se o contexto fornecido **não contiver informações suficientes para responder com precisão**, diga que não possui dados suficientes em vez de tentar adivinhar.

Cite os artigos em que as respostas foram baseadas.

### **Contexto:**
Art. 11.  As normas gerais sobre concurso público são as fixadas em lei 
específica.  
§ (V E T A D O). 
§ 2º O concurso público é de provas ou de provas e títulos, conforme dispuser 
a lei do respectivo plano de carreira.  
Art. 12.  O edital de concurso público tem de reservar vinte por cento das 
vagas para serem preenchidas por pessoa com deficiência, desprezada a parte decimal.  
§ 1º A vaga não preenchida na forma do caput reve

<ipython-input-32-f1809647c7cd>:33: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Você é um assistente especializado em recuperação de informações. Sua resposta deve ser **clara, objetiva e baseada exclusivamente nos documentos fornecidos**. **Não invente informações.** Responda **apenas em português**.

Se o contexto fornecido **não contiver informações suficientes para responder com precisão**, diga que não possui dados suficientes em vez de tentar adivinhar.

Cite os artigos em que as respostas foram baseadas.

### **Contexto:**
Art. 11.  As normas gerais sobre concurso público são as fixadas em lei 
específica.  
§ (V E T A D O). 
§ 2º O concurso público é de provas ou de provas e títulos, conforme dispuser 
a lei do respectivo plano de carreira.  
Art. 12.  O edital de concurso público tem de reservar vinte por cento das 
vagas para serem preenchidas por pessoa com deficiência, desprezada a parte decimal.  
§ 1º A vaga não preenchida na forma do caput reve

In [33]:
!pip install ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.3/178.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00


In [34]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

In [35]:
df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,Qual é o percentual aplicado pela Lei Compleme...,[Art. 11. As normas gerais sobre concurso púb...,O percentual aplicado pela Lei Complementar 84...,O percentual aplicado para reserva de vagas às...,1.000000,1.000000,1.000000,0.987670
1,A realização de concurso público é obrigatório...,[Art. 11. As normas gerais sobre concurso púb...,A realização de concurso público é obrigatória...,A realização de concurso público é obrigatório...,0.866667,1.000000,1.000000,0.932273
2,Quais fatores são observados durante o estágio...,"[probatório, observado, no mínimo, o seguinte:...",Durante o estágio probatório de um servidor pú...,"São observadas a responsabilidade, a assiduida...",1.000000,1.000000,0.857143,1.000000
3,Tenho direito a folga para consulta ou exames ...,"[II – sofrer compensação de mora, na forma da ...",Não possuo dados suficientes para responder a ...,"Sem prejuízo da remuneração ou subsídio, o ser...",0.887500,1.000000,0.000000,0.000000
4,"Para fazer mestrado ou doutorado, tenho direit...","[ou mediante compensação de horário, afastar -...",Para participar de programas de mestrado ou do...,Pode ser concedido horário especial ao servido...,1.000000,1.000000,1.000000,0.000000
5,"Fui aprovado em concurso público federal, tend...",[I – as férias; \nII – as ausências previstas...,Não possuo dados suficientes para responder a ...,Pode gozar de afastamento caso haja expressa p...,1.000000,0.000000,0.000000,0.000000
6,O que acontece se eu acumular dois cargos públ...,"[quando houver compatibilidade de horários, pa...",Se você acumular dois cargos públicos de forma...,A Lei Complementar nº 840/2011 permite a acumu...,1.000000,0.666667,1.000000,0.949737
7,"Estou próximo a me aposentar, tenho alguma res...","[ou aposentadoria, as férias não gozadas são i...",Não possuo dados suficientes para responder a ...,Algumas possíveis restrições: a) é necessário ...,0.679167,0.000000,0.000000,0.000000
8,Posso ter um outro trabalho na esfera privada?,"[autarquia ou fundação do Distrito Federal, o ...",Não possuo dados suficientes para responder a ...,A Lei Complementar nº 840/2011 não proíbe o ex...,1.000000,1.000000,0.000000,0.000000
9,Como é a licença prêmio/servidor?,[pena de cancelamento da licença. \n§ 3º (V E ...,A licença-prêmio por assiduidade é um direito ...,A licença prêmio foi substituída pela licença ...,0.679167,0.000000,0.888889,0.912079


In [36]:
df.to_excel("OpenAI_RAG_Chroma_LC840.xlsx", index=False)